# 如何对CSV文件进行问答
大型语言模型（LLMs）非常适合用于构建基于各类数据源的问答系统。本节将介绍如何针对存储在CSV文件中的数据进行问答系统开发。与操作SQL数据库类似，处理CSV文件的关键在于让LLM具备查询和交互数据的工具访问权限。主要实现方式有两种：
* **推荐**：将CSV文件导入SQL数据库，并采用[SQL教程](/docs/tutorials/sql_qa)中概述的方法进行操作。* 让LLM能够访问一个Python环境，在此环境中它可以使用Pandas等库来与数据进行交互。
在本指南中，我们将涵盖这两种方法。
## ⚠️ 安全提示 ⚠️
上述两种方法均存在重大风险。使用SQL需要执行模型生成的SQL查询语句，而使用类似Pandas的库则需允许模型执行Python代码。由于严格限制SQL连接权限和净化SQL查询语句比构建Python沙箱环境更为容易，**我们强烈建议通过SQL与CSV数据进行交互**。更多通用安全实践指南，请[参阅此处](/docs/security)。

## 安装设置本指南的依赖项：

In [ ]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

设置必需的环境变量：

In [1]:
# Using LangSmith is recommended but not required. Uncomment below lines to use.
# import os
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

下载 [泰坦尼克号数据集](https://www.kaggle.com/datasets/yasserh/titanic-dataset)（如果您尚未拥有该数据集）：

In [ ]:
!wget https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv -O titanic.csv

In [1]:
import pandas as pd

df = pd.read_csv("titanic.csv")
print(df.shape)
print(df.columns.tolist())

(887, 8)
['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']


## SQL
使用SQL与CSV数据交互是推荐的方法，因为相比任意的Python代码，它更容易限制权限和净化查询。
大多数SQL数据库都能轻松将CSV文件加载为表（例如[DuckDB](https://duckdb.org/docs/data/csv/overview.html)、[SQLite](https://www.sqlite.org/csv.html)等）。完成此操作后，您就可以使用[SQL教程](/docs/tutorials/sql_qa)中概述的所有链和代理创建技术。以下是一个使用SQLite实现的快速示例：

In [2]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///titanic.db")
df.to_sql("titanic", engine, index=False)

887

In [3]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM titanic WHERE Age < 2;"))

sqlite
['titanic']
[(1, 2, 'Master. Alden Gates Caldwell', 'male', 0.83, 0, 2, 29.0), (0, 3, 'Master. Eino Viljami Panula', 'male', 1.0, 4, 1, 39.6875), (1, 3, 'Miss. Eleanor Ileen Johnson', 'female', 1.0, 1, 1, 11.1333), (1, 2, 'Master. Richard F Becker', 'male', 1.0, 2, 1, 39.0), (1, 1, 'Master. Hudson Trevor Allison', 'male', 0.92, 1, 2, 151.55), (1, 3, 'Miss. Maria Nakid', 'female', 1.0, 0, 2, 15.7417), (0, 3, 'Master. Sidney Leonard Goodwin', 'male', 1.0, 5, 2, 46.9), (1, 3, 'Miss. Helene Barbara Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 3, 'Miss. Eugenie Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 2, 'Master. Viljo Hamalainen', 'male', 0.67, 1, 1, 14.5), (1, 3, 'Master. Bertram Vere Dean', 'male', 1.0, 1, 2, 20.575), (1, 3, 'Master. Assad Alexander Thomas', 'male', 0.42, 0, 1, 8.5167), (1, 2, 'Master. Andre Mallet', 'male', 1.0, 0, 2, 37.0042), (1, 2, 'Master. George Sibley Richards', 'male', 0.83, 1, 1, 18.75)]


并创建一个 [SQL 代理](/docs/tutorials/sql_qa) 与之交互：
import ChatModelTabs from "@theme/ChatModelTabs";
<ChatModelTabs customVarName="llm" />


In [6]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [7]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [8]:
agent_executor.invoke({"input": "what's the average age of survivors"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


titanic
Invoking: `sql_db_schema` with `{'table_names': 'titanic'}`



CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"Siblings/Spouses Aboard" BIGINT, 
	"Parents/Children Aboard" BIGINT, 
	"Fare" FLOAT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings/Spouses Aboard	Parents/Children Aboard	Fare
0	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/
Invoking: `sql_db_query` with `{'query': 'SELECT AVG(Age) AS Average_Age FROM titanic WHERE Survived = 1'}`


[(28.408391812865496,)]The average age of survivors in the Titanic dataset is approximately 28.41 years.

> Finished chain.


{'input': "what's the average age of survivors",
 'output': 'The average age of survivors in the Titanic dataset is approximately 28.41 years.'}

这种方法可以轻松推广到处理多个CSV文件，因为我们只需将每个文件作为独立的表加载到数据库中即可。具体操作请参阅下方的[多CSV文件处理](/docs/how_to/sql_csv#multiple-csvs)章节。

## Pandas
除了SQL，我们还可以使用像pandas这样的数据分析库和LLM的代码生成能力来与CSV数据交互。再次强调，**除非你已部署全面的保障措施，否则该方法不适合生产环境用例**。因此，我们的代码执行工具和构造函数都存放在`langchain-experimental`包中。
### 链
大多数大型语言模型（LLM）已经接受了足够的pandas Python代码训练，只需简单提示即可生成相应代码。

In [9]:
ai_msg = llm.invoke(
    "I have a pandas DataFrame 'df' with columns 'Age' and 'Fare'. Write code to compute the correlation between the two columns. Return Markdown for a Python code snippet and nothing else."
)
print(ai_msg.content)

```python
correlation = df['Age'].corr(df['Fare'])
correlation
```


我们可以将这一能力与执行Python的工具相结合，创建一个简单的数据分析链。首先需要将CSV表格作为数据框加载，并为该工具提供访问此数据框的权限：

In [10]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

df = pd.read_csv("titanic.csv")
tool = PythonAstREPLTool(locals={"df": df})
tool.invoke("df['Fare'].mean()")

32.30542018038331

为了确保正确使用我们的Python工具，我们将采用[工具调用](/docs/how_to/tool_calling)功能：

In [11]:
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)
response = llm_with_tools.invoke(
    "I have a dataframe 'df' and want to know the correlation between the 'Age' and 'Fare' columns"
)
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SBrK246yUbdnJemXFC8Iod05', 'function': {'arguments': '{"query":"df.corr()[\'Age\'][\'Fare\']"}', 'name': 'python_repl_ast'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 125, 'total_tokens': 138}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-1fd332ba-fa72-4351-8182-d464e7368311-0', tool_calls=[{'name': 'python_repl_ast', 'args': {'query': "df.corr()['Age']['Fare']"}, 'id': 'call_SBrK246yUbdnJemXFC8Iod05'}])

In [12]:
response.tool_calls

[{'name': 'python_repl_ast',
  'args': {'query': "df.corr()['Age']['Fare']"},
  'id': 'call_SBrK246yUbdnJemXFC8Iod05'}]

我们将添加一个工具输出解析器，以将函数调用提取为字典：

In [13]:
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser

parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)
(llm_with_tools | parser).invoke(
    "I have a dataframe 'df' and want to know the correlation between the 'Age' and 'Fare' columns"
)

{'query': "df[['Age', 'Fare']].corr()"}

并结合提示，这样我们只需指定问题而无需在每次调用时都指定数据框信息：

In [14]:
system = f"""You have access to a pandas dataframe `df`. \
Here is the output of `df.head().to_markdown()`:

```
{df.head().to_markdown()}
```

Given a user question, write the Python code to answer it. \
Return ONLY the valid Python code and nothing else. \
Don't assume you have access to any libraries other than built-in Python ones and pandas."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])
code_chain = prompt | llm_with_tools | parser
code_chain.invoke({"question": "What's the correlation between age and fare"})

{'query': "df[['Age', 'Fare']].corr()"}

最后，我们将添加我们的 Python 工具，以便实际执行生成的代码：

In [15]:
chain = prompt | llm_with_tools | parser | tool
chain.invoke({"question": "What's the correlation between age and fare"})

0.11232863699941621

就这样，我们构建了一个简单的数据分析链。通过查看LangSmith跟踪记录，我们可以一窥中间步骤的详情：https://smith.langchain.com/public/b1309290-7212-49b7-bde2-75b39a32b49a/r
我们可以在最后增加一次额外的LLM调用来生成对话式回应，这样我们就不仅仅是返回工具的输出结果了。为此，我们需要在提示词模板中添加一个聊天历史记录的`MessagesPlaceholder`占位符：

In [16]:
from operator import itemgetter

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

system = f"""You have access to a pandas dataframe `df`. \
Here is the output of `df.head().to_markdown()`:

```
{df.head().to_markdown()}
```

Given a user question, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas.
Respond directly to the question once you have enough information to answer it."""
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system,
        ),
        ("human", "{question}"),
        # This MessagesPlaceholder allows us to optionally append an arbitrary number of messages
        # at the end of the prompt using the 'chat_history' arg.
        MessagesPlaceholder("chat_history", optional=True),
    ]
)


def _get_chat_history(x: dict) -> list:
    """Parse the chain output up to this point into a list of chat history messages to insert in the prompt."""
    ai_msg = x["ai_msg"]
    tool_call_id = x["ai_msg"].additional_kwargs["tool_calls"][0]["id"]
    tool_msg = ToolMessage(tool_call_id=tool_call_id, content=str(x["tool_output"]))
    return [ai_msg, tool_msg]


chain = (
    RunnablePassthrough.assign(ai_msg=prompt | llm_with_tools)
    .assign(tool_output=itemgetter("ai_msg") | parser | tool)
    .assign(chat_history=_get_chat_history)
    .assign(response=prompt | llm | StrOutputParser())
    .pick(["tool_output", "response"])
)

In [17]:
chain.invoke({"question": "What's the correlation between age and fare"})

{'tool_output': 0.11232863699941616,
 'response': 'The correlation between age and fare is approximately 0.1123.'}

本次运行的LangSmith追踪记录如下：  
https://smith.langchain.com/public/14e38d70-45b1-4b81-8477-9fd2b7c07ea6/r

### 代理
在处理复杂问题时，大型语言模型（LLM）若能迭代执行代码并保留先前执行的输入输出记录将大有裨益。这正是智能体（Agents）的用武之地——它们使LLM能够自主决定工具调用次数，并持续追踪已执行的操作。[create_pandas_dataframe_agent](https://python.langchain.com/api_reference/experimental/agents/langchain_experimental.agents.agent_toolkits.pandas.base.create_pandas_dataframe_agent.html)作为内置智能体，可显著简化数据框操作流程：

In [18]:
from langchain_experimental.agents import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(
    llm, df, agent_type="openai-tools", verbose=True, allow_dangerous_code=True
)
agent.invoke(
    {
        "input": "What's the correlation between age and fare? is that greater than the correlation between fare and survival?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[['Age', 'Fare']].corr().iloc[0,1]"}`


0.11232863699941621
Invoking: `python_repl_ast` with `{'query': "df[['Fare', 'Survived']].corr().iloc[0,1]"}`


0.2561785496289603The correlation between Age and Fare is approximately 0.112, and the correlation between Fare and Survival is approximately 0.256.

Therefore, the correlation between Fare and Survival (0.256) is greater than the correlation between Age and Fare (0.112).

> Finished chain.


{'input': "What's the correlation between age and fare? is that greater than the correlation between fare and survival?",
 'output': 'The correlation between Age and Fare is approximately 0.112, and the correlation between Fare and Survival is approximately 0.256.\n\nTherefore, the correlation between Fare and Survival (0.256) is greater than the correlation between Age and Fare (0.112).'}

这是本次运行的LangSmith追踪记录：[点击查看](https://smith.langchain.com/public/6a86aee2-4f22-474a-9264-bd4c7283e665/r)

### 多个CSV文件 {#multiple-csvs}
要处理多个CSV文件（或数据框），我们只需向Python工具传递多个数据框即可。`create_pandas_dataframe_agent`构造函数原生支持此功能——我们可以直接传入一个数据框列表，而不仅限于单个数据框。若需自行构建处理链，可采用如下方式：

In [19]:
df_1 = df[["Age", "Fare"]]
df_2 = df[["Fare", "Survived"]]

tool = PythonAstREPLTool(locals={"df_1": df_1, "df_2": df_2})
llm_with_tool = llm.bind_tools(tools=[tool], tool_choice=tool.name)
df_template = """```python
{df_name}.head().to_markdown()
>>> {df_head}
```"""
df_context = "\n\n".join(
    df_template.format(df_head=_df.head().to_markdown(), df_name=df_name)
    for _df, df_name in [(df_1, "df_1"), (df_2, "df_2")]
)

system = f"""You have access to a number of pandas dataframes. \
Here is a sample of rows from each dataframe and the python code that was used to generate the sample:

{df_context}

Given a user question about the dataframes, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas. \
Make sure to refer only to the variables mentioned above."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])

chain = prompt | llm_with_tool | parser | tool
chain.invoke(
    {
        "question": "return the difference in the correlation between age and fare and the correlation between fare and survival"
    }
)

0.14384991262954416

本次运行的LangSmith追踪记录如下：  
https://smith.langchain.com/public/cc2a7d7f-7c5a-4e77-a10c-7b5420fcd07f/r

### 沙盒化代码执行
有多款工具（如 [E2B](/docs/integrations/tools/e2b_data_analysis) 和 [Bearly](/docs/integrations/tools/bearly)）提供了 Python 代码执行的沙盒环境，以实现更安全的代码执行链和代理。

## 后续步骤
对于更高级的数据分析应用，我们推荐您查阅：
* [SQL教程](/docs/tutorials/sql_qa)：处理SQL数据库和CSV文件时的许多挑战对于任何结构化数据类型都是通用的，因此即使您使用Pandas进行CSV数据分析，阅读SQL技术也很有帮助。* [工具使用](/docs/how_to/tool_calling): 关于调用工具的链和代理的通用最佳实践指南* [智能体](/docs/tutorials/agents): 了解构建大语言模型智能体的基础知识。* 集成功能：沙盒环境如 [E2B](/docs/integrations/tools/e2b_data_analysis) 和 [Bearly](/docs/integrations/tools/bearly)，实用工具如 [SQLDatabase](https://python.langchain.com/api_reference/community/utilities/langchain_community.utilities.sql_database.SQLDatabase.html#langchain_community.utilities.sql_database.SQLDatabase)，相关代理如 [Spark DataFrame 代理](/docs/integrations/tools/spark_sql)。